## Imports

In [2]:
import glob
import os
import xml.etree.ElementTree as ET
from PIL import Image
import math

# Rotating Images using Imgaug Library

In [7]:
import imageio
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 


image_path = 'K:/Study/Semester 7/DS/Project/PyCharm Folder/Pascal VOC Files/CNIC/JPEGImages'
annotation_path = 'K:/Study/Semester 7/DS/Project/PyCharm Folder/Pascal VOC Files/CNIC/Annotations'
xml_destination_path = 'K:/Study/Semester 7/DS/Project/PyCharm Folder/Rotated_CNIC/Rotated_XMLs'
image_destination_path = 'K:/Study/Semester 7/DS/Project/PyCharm Folder/Rotated_CNIC/Rotated_Images/'

i = 1

for file in os.listdir(annotation_path):
    fullname = os.path.join(annotation_path, file) 
    tree = ET.parse(fullname)
    og_root = tree.getroot()
    
    image_file_name = og_root.find('filename').text
    
    print(image_file_name)
    
    # opening corresponding image file
    image_directory = os.path.join(image_path, image_file_name) 
    image = imageio.imread(image_directory)

    degree = 0
    rotation_num = 0
    
    while degree < 360:
        ia.seed(1)
        
        # making a copy of original xml file
        # converting xml to string
        xml_string = ET.tostring(og_root)

        # now parsing xml from that string instead of xml file
        # to prevent the original file from being updated
        
        # .fromstring returns element, NOT an element tree
        element = ET.fromstring(xml_string)
        # converting element to element tree
        tree = ET.ElementTree(element)
        root = tree.getroot()
    
        bb_list = []

        for coordinate in og_root.findall('object/bndbox'):
            xmin = float(coordinate.find('xmin').text)
            ymin = float(coordinate.find('ymin').text)
            xmax = float(coordinate.find('xmax').text)
            ymax = float(coordinate.find('ymax').text)

            bb = [xmin, ymin, xmax, ymax]
            bb_list.append(bb)
        
        
        bbs = BoundingBoxesOnImage([
            BoundingBox(bb_list[0][0], bb_list[0][1], bb_list[0][2], bb_list[0][3]),
            BoundingBox(bb_list[1][0], bb_list[1][1], bb_list[1][2], bb_list[1][3]),
            BoundingBox(bb_list[2][0], bb_list[2][1], bb_list[2][2], bb_list[2][3]),
            BoundingBox(bb_list[3][0], bb_list[3][1], bb_list[3][2], bb_list[3][3]),
            BoundingBox(bb_list[4][0], bb_list[4][1], bb_list[4][2], bb_list[4][3]),
        ], shape=image.shape)
        
        
        
        # rotating image and the bounding boxes in it
        image_aug, bbs_aug = iaa.Affine(rotate=degree)(image=image, bounding_boxes=bbs)
        # ia.imshow(bbs_aug.draw_on_image(image_aug))

        bb_index = 0
        for coordinate in root.findall('object/bndbox'):
            coordinate.find('.//xmin').text = str(bbs_aug[bb_index][0][0])
            coordinate.find('.//ymin').text = str(bbs_aug[bb_index][0][1])
            coordinate.find('.//xmax').text = str(bbs_aug[bb_index][1][0])
            coordinate.find('.//ymax').text = str(bbs_aug[bb_index][1][0])
            bb_index += 1
        
        # converting updated xml to string
        xml_string = ET.tostring(root)
        
#         # storing XMLs
#         with open(os.path.join(xml_destination_path, image_file_name.split('.')[0] + '_rotation' + str(rotation_num) + '.xml'), "wb") as f:
#             f.write(xml_string)
#             f.close()

#         # storing image
#         file_name = image_file_name.split('.')[0] + '_rotation' + str(rotation_num) + '.png'
        
#         imageio.imwrite(image_destination_path + file_name, image_aug)
        
        degree += 4
        rotation_num += 1
        
    if i==1:
        break
        

In [85]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))